<a href="https://colab.research.google.com/github/danielmlow/tutorials/blob/main/text/keyword_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keybert import KeyBERT

!pip install -q git+https://github.com/LIAAD/yake


/Users/alina/opt/anaconda3/envs/daniel/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

import yake
import pandas as pd
import numpy as np
from pathlib import Path 
from collections import OrderedDict

In [5]:
data_root = Path("/Volumes/Samsung_T5/MIT/abcd/release_05/abcd-data-release-5.1")
vocab_path  = Path("/Volumes/Samsung_T5/MIT/abcd/adhd_vocab.npy")

section = "core/mental-health"
table = 'mh_p_ksads_adhd.csv'
mapping = 'ksads_adhd.csv'

data_section = Path( data_root, section)
data_path = Path(data_section, table) # path to results table  
var_mapping_path =  Path(data_root, "var_mapping", mapping) # path to mapping of variable names to questions 

data = pd.read_csv(data_path)
var_description = pd.read_csv(var_mapping_path)
vocab = np.load(vocab_path, allow_pickle=True)

var_names = np.array(var_description["var_name"])
questions = np.array(var_description["var_label"])
mapping = OrderedDict(zip(var_names, questions))

to_map = vocab

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_94318/1561401651.py:12: DtypeWarning: Columns (23,76,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_path)


In [6]:
kw_model = KeyBERT(model='all-MiniLM-L6-v2')

In [55]:
vocab

array(['irritability', 'stress generation', 'agression', 'anhedhonia',
       'Reward anticipation', 'Addiction ', 'delay aversion (impatience)',
       'vigilance', 'heigthend awareness/sensitivity (sensory overleod) ',
       'heigthend awareness/sensitivity (sensory overleod) ',
       'heigthend awareness/sensitivity (sensory overleod) ',
       'Impulse control ', 'disruptive behavior', 'distractability',
       'Impulse control ', 'cognitive inhibition ',
       'planning and organisation', 'vigilance', 'set-shifting',
       'cognitive flexibility', 'Timing', 'executive control',
       'social cognition', 'arousal moderation ', 'unstable sleep habits',
       'fine motor coordination', 'motor inhibition ', 'DSM-5',
       'I : inattentive type'], dtype=object)

In [17]:
embeddings_qst = kw_model.extract_embeddings(questions)
embeddings_vocab = kw_model.extract_embeddings(vocab)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
embeddings_qst[0].shape

(25, 384)

In [40]:
cos_sim = cosine_similarity(embeddings_qst[0],embeddings_vocab[0])
cos_sim.shape

(25, 29)

In [50]:
cos_sim[np.where(cos_sim[0] == np.max(cos_sim[0]))]

array([[ 0.11046764,  0.05110805, -0.03909838,  0.11266285,  0.14853942,
         0.1420542 ,  0.07002442,  0.08416943,  0.00579727,  0.00579727,
         0.00579727, -0.06929769,  0.10861532,  0.1728814 , -0.06929775,
         0.09467658,  0.05815695,  0.08416943,  0.02364541,  0.15135919,
         0.06376207,  0.0779245 ,  0.06910644, -0.01397495,  0.19049473,
         0.00302587, -0.01266796,  0.04308222,  0.14602   ]],
      dtype=float32)

In [51]:
for i, qst in enumerate(questions):
    print('Questions:' ,qst)
    highest_score = np.max(cos_sim[i])
    best_word = vocab[np.where(cos_sim[i] == np.max(cos_sim[i]))]
    print(best_word, highest_score)
    

Questions: In the past two weeks, how often has your child put off or avoided doing things that were hard for him or her because they required a lot of attention?
['disruptive behavior'] 0.36555764
Questions: Do the problems we have just been talking about cause difficulties for your child in any of the following areas:With family
['disruptive behavior'] 0.26917803
Questions: Do the problems we have just been talking about cause difficulties for your child in any of the following areas:With friends
['disruptive behavior'] 0.3214041
Questions: In the past two weeks, how often has your child talked when he or she wasn't supposed to, or had people complain that he or she talked too much?
['disruptive behavior'] 0.26132447
Questions: In the past two weeks, how often has your child interrupted someone when they were talking?
['disruptive behavior'] 0.4102337
Questions: In the past two weeks, how often has your child had trouble waiting for his or her turn or waiting for other things?
['dela

In [56]:
kw = kw_model.extract_keywords(questions, seed_keywords=vocab)


In [57]:
#kw_model = KeyBERT()
kw

[[('child', 0.3498),
  ('attention', 0.3061),
  ('avoided', 0.3044),
  ('weeks', 0.2677),
  ('things', 0.1766)],
 [('difficulties', 0.5338),
  ('problems', 0.4864),
  ('family', 0.3447),
  ('child', 0.3289),
  ('areas', 0.2439)],
 [('difficulties', 0.5201),
  ('problems', 0.4804),
  ('child', 0.3708),
  ('talking', 0.2471),
  ('areas', 0.219)],
 [('child', 0.3444),
  ('weeks', 0.262),
  ('talked', 0.2501),
  ('complain', 0.2301),
  ('past', 0.1315)],
 [('interrupted', 0.4502),
  ('talking', 0.3956),
  ('child', 0.3852),
  ('weeks', 0.2439),
  ('past', 0.1926)],
 [('child', 0.3426),
  ('waiting', 0.3122),
  ('weeks', 0.2672),
  ('turn', 0.2003),
  ('things', 0.1516)],
 [('child', 0.3568),
  ('talking', 0.3392),
  ('answers', 0.3023),
  ('school', 0.2993),
  ('blurt', 0.2846)],
 [('quiet', 0.4319),
  ('child', 0.3331),
  ('weeks', 0.2507),
  ('past', 0.1215),
  ('trouble', 0.0977)],
 [('moving', 0.4618),
  ('child', 0.3811),
  ('weeks', 0.3348),
  ('place', 0.1953),
  ('past', 0.185)],
 

In [ ]:
print(keywords)

['irritability' 'stress generation' 'agression' 'anhedhonia'
 'Reward anticipation' 'Addiction ' 'delay aversion (impatience)'
 'vigilance' 'heigthend awareness/sensitivity (sensory overleod) '
 'heigthend awareness/sensitivity (sensory overleod) '
 'heigthend awareness/sensitivity (sensory overleod) ' 'Impulse control '
 'disruptive behavior' 'distractability' 'Impulse control '
 'cognitive inhibition ' 'planning and organisation' 'vigilance'
 'set-shifting' 'cognitive flexibility' 'Timing' 'executive control'
 'social cognition' 'arousal moderation ' 'unstable sleep habits'
 'fine motor coordination' 'motor inhibition ' 'DSM-5'
 'I : inattentive type']


In [15]:

language = "en"
max_ngram_size = 1
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 10

for t in questions:
    print('='*80)
    print()
    print("title from which we'll extract keywords:", t)
    for model in ['keybert', 'yake']:
      for ngram_range_max in [1,2]:
        if model == 'keybert':
          print(f'model: {model}      ngram_range_max: {ngram_range_max}')
          kw = kw_model.extract_keywords(t, keyphrase_ngram_range=(1,ngram_range_max), stop_words='english',
                                        #  highlight=True,
                                    top_n=10,
                                    # use_maxsum=False, nr_candidates=20, 
                                    # use_mmr=False, diversity=0.7
                                    )
          [print(k[0], np.round(k[1],3)) for k in kw]
          print()
        
        elif model == 'yake':
          print(f'model: {model}      ngram_range_max: {ngram_range_max}')
          custom_kw_extractor = yake.KeywordExtractor(lan=language, n=ngram_range_max, )#dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
          keywords = custom_kw_extractor.extract_keywords(t)

          for kw in keywords:
            print(kw[0], np.round(kw[1],3))
          print()
  


title from which we'll extract keywords: In the past two weeks, how often has your child put off or avoided doing things that were hard for him or her because they required a lot of attention?
model: keybert      ngram_range_max: 1
child 0.31
weeks 0.255
avoided 0.235
attention 0.235
past 0.118
things 0.112
doing 0.09
lot 0.09
hard 0.064
required 0.047

model: keybert      ngram_range_max: 2
child avoided 0.53
weeks child 0.34
avoided doing 0.316
child 0.31
lot attention 0.283
weeks 0.255
avoided 0.235
attention 0.235
past weeks 0.231
doing things 0.227

model: yake      ngram_range_max: 1
weeks 0.096
attention 0.096
past 0.158
child 0.158
put 0.158
avoided 0.158
things 0.158
hard 0.158
required 0.158
lot 0.158

model: yake      ngram_range_max: 2
child put 0.026
weeks 0.096
attention 0.096
past 0.158
child 0.158
put 0.158
avoided 0.158
things 0.158
hard 0.158
required 0.158
lot 0.158


title from which we'll extract keywords: Do the problems we have just been talking about cause diff

### For scientific articles
I decided to extract:
- KeyBert Title (1,1) top 3
- KeyBert Title+Abstract (1,1) top 7
- KeyBert Title (1,2) top 3
'''